In [1]:
import numpy as np

# 定義欲積分的函數
def f(x):
    return np.sin(4 * x) * np.exp(x)

# 積分範圍與步長
a = 1
b = 2
h = 0.1
n = int((b - a) / h)  # 區間個數

# Composite Trapezoidal Rule
def trapezoidal_rule(f, a, b, h):
    x = np.arange(a, b + h, h)
    y = f(x)
    return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

# Composite Simpson’s Rule
def simpsons_rule(f, a, b, h):
    if (b - a) / h % 2 != 0:
        raise ValueError("Simpson's rule requires an even number of intervals.")
    x = np.arange(a, b + h, h)
    y = f(x)
    return (h / 3) * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]) + y[-1])

# Composite Midpoint Rule
def midpoint_rule(f, a, b, h):
    midpoints = np.arange(a + h/2, b, h)
    return h * np.sum(f(midpoints))

# 計算三種方法的近似積分值
trap_result = trapezoidal_rule(f, a, b, h)
simpson_result = simpsons_rule(f, a, b, h)
midpoint_result = midpoint_rule(f, a, b, h)

# 顯示結果
print(f"Trapezoidal Rule Result: {trap_result:.8f}")
print(f"Simpson's Rule Result:   {simpson_result:.8f}")
print(f"Midpoint Rule Result:    {midpoint_result:.8f}")


Trapezoidal Rule Result: 0.39614759
Simpson's Rule Result:   0.38566360
Midpoint Rule Result:    0.38080480


In [3]:
import numpy as np
from scipy.integrate import quad

# 原始函數
def f(x):
    return x * np.log(x)

# 積分區間
a = 1
b = 1.5

# 變數變換後的新函數（用 t 表示）
def transformed_f(t):
    x = 0.25 * t + 1.25  # 將 t 映射回 x
    return f(x)

# Gaussian weights and nodes for n=3 and n=4
gauss_nodes_weights = {
    3: {
        "nodes": np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)]),
        "weights": np.array([5/9, 8/9, 5/9])
    },
    4: {
        "nodes": np.array([-0.8611363116, -0.3399810436, 0.3399810436, 0.8611363116]),
        "weights": np.array([0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451])
    }
}

def gaussian_quadrature(n):
    nodes = gauss_nodes_weights[n]["nodes"]
    weights = gauss_nodes_weights[n]["weights"]
    integral = 0.0
    for t, w in zip(nodes, weights):
        integral += w * transformed_f(t)
    # 別忘了乘上縮放因子 (b - a)/2
    return (b - a)/2 * integral

# 使用 scipy 計算精確積分值
exact_value, _ = quad(f, a, b)

# 計算 Gaussian Quadrature 近似值
gq3 = gaussian_quadrature(3)
gq4 = gaussian_quadrature(4)

# 輸出結果
print(f"Exact value:             {exact_value:.10f}")
print(f"Gaussian Quadrature (n=3): {gq3:.10f}")
print(f"Gaussian Quadrature (n=4): {gq4:.10f}")


Exact value:             0.1436482466
Gaussian Quadrature (n=3): 0.1436482150
Gaussian Quadrature (n=4): 0.1436482464


In [5]:
import numpy as np
from scipy.integrate import dblquad

# 原始函數
def f(x, y):
    return y / (1 + 2 * np.sin(x) * np.cos(x))

# 積分範圍
a, b = 0, np.pi / 4

# Simpson's rule 二重積分
def simpsons_double(f, a, b, n, m):
    hx = (b - a) / n
    result = 0

    for i in range(n + 1):
        x = a + i * hx
        cx = 1 if i == 0 or i == n else (4 if i % 2 == 1 else 2)

        y_lower = np.sin(2 * x)
        y_upper = np.cos(x) ** 2
        hy = (y_upper - y_lower) / m

        inner_sum = 0
        for j in range(m + 1):
            y = y_lower + j * hy
            cy = 1 if j == 0 or j == m else (4 if j % 2 == 1 else 2)
            inner_sum += cy * f(x, y)

        result += cx * (hy / 3) * inner_sum

    return (hx / 3) * result

# Gaussian Quadrature 用 3 點 (x,y 都是)
def gaussian_double(f, a, b, n, m):
    # 高斯節點與權重（標準區間 [-1,1]）
    nodes = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
    weights = np.array([5/9, 8/9, 5/9])

    total = 0
    for i in range(n):
        # x 範圍切成 n 段
        x0 = a + i * (b - a) / n
        x1 = a + (i + 1) * (b - a) / n
        for xi, wi in zip(nodes, weights):
            x = (x1 - x0) / 2 * xi + (x1 + x0) / 2
            wx = (x1 - x0) / 2 * wi

            y_lower = np.sin(2 * x)
            y_upper = np.cos(x) ** 2

            for j in range(m):
                y0 = y_lower + j * (y_upper - y_lower) / m
                y1 = y_lower + (j + 1) * (y_upper - y_lower) / m
                for yj, wj in zip(nodes, weights):
                    y = (y1 - y0) / 2 * yj + (y1 + y0) / 2
                    wy = (y1 - y0) / 2 * wj

                    total += wx * wy * f(x, y)

    return total

# 精確值（用 scipy 的 dblquad）
exact, _ = dblquad(f, a, b, lambda x: np.sin(2 * x), lambda x: np.cos(x) ** 2)

# 套用兩種方法
simpson_result = simpsons_double(f, a, b, n=4, m=4)
gauss_result = gaussian_double(f, a, b, n=3, m=3)

# 顯示結果
print(f"Exact value:                {exact:.10f}")
print(f"Simpson’s Rule (n=4, m=4):  {simpson_result:.10f}")
print(f"Gaussian Quad (n=3, m=3):   {gauss_result:.10f}")


Exact value:                -0.0118013564
Simpson’s Rule (n=4, m=4):  0.0773974189
Gaussian Quad (n=3, m=3):   0.0775173265


In [13]:
import numpy as np

# Composite Simpson's Rule
def composite_simpsons(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)

    result = y[0] + y[-1]
    result += 4 * np.sum(y[1:-1:2])
    result += 2 * np.sum(y[2:-2:2])
    return (h / 3) * result

# === (a) ===
# f(x) = x^(-1/4) * sin(x)
def f1(x):
    return np.where(x == 0, 0, x**(-1/4) * np.sin(x))

a1 = 0
b1 = 1
n1 = 4
result_a = composite_simpsons(f1, a1, b1, n1)

# === (b) ===
# Use substitution: x = 1 / (1 - t), so dx/dt = 1 / (1 - t)^2
# New limits: x = 1 → t = 0; x = ∞ → t = 1
def f2_transformed(t):
    with np.errstate(divide='ignore', invalid='ignore'):
        x = 1 / (1 - t)
        dxdt = 1 / (1 - t)**2
        result = (x**-4) * np.sin(x) * dxdt
        result = np.where((t >= 1), 0, result)  # 處理無窮大處的數值不穩定
        return result

a2 = 0
b2 = 1
n2 = 4
result_b = composite_simpsons(f2_transformed, a2, b2, n2)

# 顯示結果
print(f"(a) ∫₀¹ x^(-1/4) sin(x) dx ≈ {result_a:.10f}")
print(f"(b) ∫₁^∞ x^(-4) sin(x) dx ≈ {result_b:.10f}")


(a) ∫₀¹ x^(-1/4) sin(x) dx ≈ 0.5259288092
(b) ∫₁^∞ x^(-4) sin(x) dx ≈ 0.2744816127


C:\Users\王子豪\AppData\Local\Temp\ipykernel_29568\3361337977.py:17: RuntimeWarning: divide by zero encountered in power
  return np.where(x == 0, 0, x**(-1/4) * np.sin(x))
C:\Users\王子豪\AppData\Local\Temp\ipykernel_29568\3361337977.py:17: RuntimeWarning: invalid value encountered in multiply
  return np.where(x == 0, 0, x**(-1/4) * np.sin(x))
